In [ ]:
from bt_gen_model import createSchema, createDataModel
import os

Schema Generation
* `createSchema` will recurse through a search path and find all JSON files with a specific prefix and exclude those with a rejection substring. In this example, we parse all the BattleTech JSONs in RogueTech, and exclude merged JSONS.

In [ ]:
# Directory to search for schemas. Define the json prefix and rejection strings prior to use.
search_dir = 'E:/Game/RtlCache/RtCache//**/*.json'

# Data model generation also uses this folder
schema_folder="schema/"

def createSchemas(search_dir, out_dir="schema/"):
    createSchema(search_dir, "mechdef_", "Merge", out_dir, False)
    #createSchema(search_dir, "Ammo_AmmunitionBox_", "Merge", out_dir, False)
    #createSchema(search_dir, "chassisdef_", "Merge", out_dir, False)
    #createSchema(search_dir, "hardpointdatadef_", "Merge", out_dir, False)
    #createSchema(search_dir, "vehicledef_", "Merge", out_dir, False)
    #createSchema(search_dir, "Weapon_", "Merge", out_dir, False)

createSchemas(search_dir, schema_folder)

Datamodel Creation
* `createDatamodel` will generate a `msgspec` struct from the a JSON schema. This resulting python file enables access to JSON data in python through nested data structures. 

In [ ]:
# Directory to create datamodels
datamodel_folder="datamodel/"

def createDataModels(path, folder="datamodel/"):
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f):
            createDataModel(f, filename, folder)

createDataModels(schema_folder) 

Data Extraction
* Now that a datamodel is created, msgspec can be used to parse JSON files matching a specified schema. In this case, we parse all `mechdefs` using the `mechdef` data model and extract them to a dataframe. This dataframe can be dumped to CSV or used for data analysis.

In [ ]:
from datamodel import mechdef, Weapon
from bt_gen_model import parseModels, extractDataframe

import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

mechdef.Model

def run(path):
    os.makedirs(os.path.dirname("csv/"), exist_ok=True)
    mechs = parseModels(path, "mechdef_", "Merge", mechdef.Model)
    mech_df = extractDataframe(mechs)
    mech_df.to_csv("mech.csv")
    
    weapons = parseModels(path, "Weapon_", "Merge", Weapon.Model)
    weapon_df= extractDataframe(weapons)
    weapon_df.to_csv("weapon.csv")
    return mech_df, weapon_df

mech_df, weapon_df = run(search_dir)

Data View
* Pandas can be used to view the dataframe and analyze the data. Add additional fields and analytics columns as necessary.

In [ ]:
from pandasgui import show as showpd
    
showpd(mech_df)